In [ ]:
# update pip
!/opt/venv/bin/python -m pip install --upgrade pip

# module installs
!pip install pandas numpy tqdm plotly sklearn tensorflow

     |████████████████████████████████| 1.5 MB 9.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3
     |████████████████████████████████| 70 kB 7.0 MB/s 
     |████████████████████████████████| 13.1 MB 18.2 MB/s 
     |████████████████████████████████| 320.4 MB 25 kB/s 
     |████████████████████████████████| 42 kB 3.1 MB/s 
     |████████████████████████████████| 65 kB 8.8 MB/s 
     |████████████████████████████████| 3.8 MB 48.2 MB/s 
     |████████████████████████████████| 2.9 MB 54.9 MB/s 
     |████████████████████████████████| 57 kB 8.7 MB/s 
     |████████████████████████████████| 127 kB 79.1 MB/s 
     |████████████████████████████████| 459 kB 61.1 MB/s 
     |████████████████████████████████| 1.3 MB 64.0 MB/s 
     |████████████████████████████████| 6.8 MB 78.0 MB/s 
     |████████████████████████████████| 298 kB 49.2 MB/s 
     |████████████████████████████████| 779 kB 76.3 M

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.graph_objects as plt
import plotly.express as px
pio.templates.default = "plotly_white"
!pwd
from sklearn import metrics

/opt/venv/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

/home/jovyan/work/sm-depression-detection


In [ ]:
NEUTRALSET = "./neutralTweets.csv"
NEUTRALROWS = 300000
DEPRESSEDSET = "./depressedTweets.csv"
DEPRESSEDROWS = 30000

In [ ]:
neutralSet = pd.read_csv(NEUTRALSET, nrows=NEUTRALROWS)
depressedSet = pd.read_csv(DEPRESSEDSET, nrows=DEPRESSEDROWS)

In [ ]:
neutralSet['target'] = 0
depressedSet['target'] = 1
mixedFinal = pd.concat([neutralSet, depressedSet], ignore_index=True)

In [ ]:
mixedFinal

,tweet,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,0
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0
...,...,...
329995,Feel like a depression cloud is forming above....,1
329996,"We have to stop talking about ""Mental Illness""...",1
329997,@tal7291 @DanLairdMD I disagree. If one has fi...,1
329998,J'vais tomber en dépression twitter.com/ojwess...,1


In [ ]:
# Expand Contractions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
# NLTK STUFF
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import EnglishStemmer
from nltk import word_tokenize

     |████████████████████████████████| 1.4 MB 13.2 MB/s 
     |████████████████████████████████| 82 kB 1.8 MB/s 
     |████████████████████████████████| 721 kB 52.4 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434673 sha256=7f05d108ac73de1e38b7652988bc4ec32da3ac59746547957c4cc02d6fae3148
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def vectandstem(x):
    tokenized = word_tokenize(str(x))
    full= " ".join(tokenized)
    return full

In [ ]:
stop_w = stopwords.words('english')
stemmer = EnglishStemmer(ignore_stopwords=True)

In [ ]:
import string
#Function to vectorize and clean text
def cleaner(x):
    # pass0 to expand the contractions
    x = expandContractions(str(x))
    # first pass to clean the punctuation
    # print(x)
    pass1  = "".join([char for char in x if char not in string.punctuation])
    # second pass to clean the numbers up
    pass2 = re.sub('[0-9]+', '', pass1)
    # third pass to tokenize
    pass3 = word_tokenize(pass2)
    # fourth pass to use snowball to stem words if they're not a stopword
    pass4 = [word for word in pass3 if word not in stop_w]
    return (pass4)

In [ ]:
mixedFinal.tweet = mixedFinal.tweet.progress_apply(cleaner)

100%|██████████| 330000/330000 [01:37<00:00, 3394.59it/s]


In [ ]:
# Hyperparameters
vocab_size = 10000
embedding_dim = 32
# idk for max length, tweets are like 280chars long
max_length = 40
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

tweet = mixedFinal.tweet.values
tokenizer = Tokenizer(num_words=vocab_size,
                          oov_token=oov_tok,
                          filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                          lower=True)
tokenizer.fit_on_texts(tweet)

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(tweet)

padded_sequences = pad_sequences(sequences, maxlen=max_length,
                                padding=padding_type,
                                truncating=trunc_type)

In [ ]:
print(tf.__version__)
print(tf.executing_eagerly())

2.3.1
True


In [ ]:
tokenizer.word_index

{'<OOV>': 1,
 'i': 2,
 'depression': 3,
 'like': 4,
 'good': 5,
 'u': 6,
 'get': 7,
 'lol': 8,
 'know': 9,
 'love': 10,
 'im': 11,
 'thanks': 12,
 'one': 13,
 'go': 14,
 'got': 15,
 'see': 16,
 'would': 17,
 'day': 18,
 'well': 19,
 'time': 20,
 'think': 21,
 'really': 22,
 'you': 23,
 'hope': 24,
 'oh': 25,
 'haha': 26,
 'going': 27,
 'back': 28,
 'sorry': 29,
 'great': 30,
 'work': 31,
 'much': 32,
 'still': 33,
 'want': 34,
 'yeah': 35,
 'na': 36,
 'need': 37,
 'today': 38,
 'could': 39,
 'right': 40,
 'the': 41,
 'miss': 42,
 'make': 43,
 'its': 44,
 'yes': 45,
 'come': 46,
 'feel': 47,
 'new': 48,
 'better': 49,
 'thank': 50,
 'though': 51,
 'hey': 52,
 'dont': 53,
 'fun': 54,
 'thats': 55,
 'wish': 56,
 'us': 57,
 'last': 58,
 'night': 59,
 'twitter': 60,
 'people': 61,
 'it': 62,
 'nice': 63,
 'bad': 64,
 'way': 65,
 'sad': 66,
 'ur': 67,
 'and': 68,
 'but': 69,
 'never': 70,
 'sure': 71,
 'even': 72,
 'happy': 73,
 'say': 74,
 'my': 75,
 'wait': 76,
 'always': 77,
 'take': 78,


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#making test train splits
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, mixedFinal['target'], test_size=0.05)

In [ ]:
print("The train set has {} and test set has {}".format(len(X_train),len(X_test)))

The train set has 313500 and test set has 16500


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',  metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 32)            10070400  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 40, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,087,051
Trainable params: 10,087,051
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Early Stopping callback setup
es = EarlyStopping(monitor='val_loss', min_delta=0.05, mode='min', patience=2, verbose=1)

# Model Checkpoint callback setup
mc = ModelCheckpoint('model_checkpoint.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[es, mc], shuffle=True)

In [ ]:
model.load_weights('model_checkpoint.h5')

In [ ]:
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)

print('Train: %.5f, Test: %.5f' % (train_acc, test_acc))

Train: 0.99880, Test: 0.99873


In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
accuracy_score(y_test, y_pred)

0.9987272727272727

In [ ]:
fpr, tpr, thresholds = roc_curve(y_pred, y_test)
roc_auc = auc(fpr, tpr)

In [ ]:
fig = plt.Figure()

fig.add_trace(plt.Scatter(
    x=fpr,
    y=tpr,
    name="ROC curve (area = {0:.4f})".format(roc_auc),
    fill = 'tozeroy'
))

fig.add_trace(plt.Scatter(
    x=[0,1],
    y=[0,1],
    name="Line",
    showlegend=False,
    line=dict(dash='dashdot')
))

fig.update_layout(
    title="ROC Curve",
    xaxis_title="FPR",
    yaxis_title="TPR",
    legend_title="",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig.show()

In [ ]:
roc_auc_score(y_test, y_pred)

0.9947833442364251

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15005
           1       1.00      0.99      0.99      1495

    accuracy                           1.00     16500
   macro avg       1.00      0.99      1.00     16500
weighted avg       1.00      1.00      1.00     16500

